<a href="https://colab.research.google.com/github/Yancyyyyyy/AMM/blob/main/AMM_Uniswap_Sushi_input_path_decode_ipynb_copy_8_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install --force-reinstall jsonschema==3.2.0 #You must restart the runtime in order to use newly installed versions.
!pip install web3
!pip install demjson
!pip3 install simplejson


     |████████████████████████████████| 56 kB 1.8 MB/s 
     |████████████████████████████████| 119 kB 8.8 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 819 kB 40.8 MB/s 
  Attempting uninstall: zipp
    Found existing installation: zipp 3.5.0
    Uninstalling zipp-3.5.0:
      Successfully uninstalled zipp-3.5.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pyrsistent
    Found existing installation: pyrsistent 0.18.0
    Uninstalling pyrsistent

     |████████████████████████████████| 485 kB 3.9 MB/s 
     |████████████████████████████████| 100 kB 10.1 MB/s 
     |████████████████████████████████| 103 kB 58.2 MB/s 
     |████████████████████████████████| 1.3 MB 49.2 MB/s 
     |████████████████████████████████| 82 kB 998 kB/s 
     |████████████████████████████████| 294 kB 73.7 MB/s 
     |████████████████████████████████| 142 kB 74.2 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████████████| 48 kB 5.0 MB/s 
     |████████████████████████████████| 1.9 MB 42.0 MB/s 
     |████████████████████████████████| 1.6 MB 14.9 MB/s 
     |████████████████████████████████| 1.9 MB 55.0 MB/s 
  Created wheel for bitarray: filename=bitarray-1.2.2-cp37-cp37m-linux_x86_64.whl size=107028 sha256=3c88b4e7f086e7bbb002ee6a467bd3ab79cacc0c0686ebd8158970a449cc1b6c
  Stored in directory: /root/.cache/pip/wheels/91/c8/a0/4fb7317b7072d4b6a4366454e5bf229ff5bb9ba1f7de8ef90d
  Created wheel for lru-dict: filename

In [1]:
# import libraries
import simplejson as json
import json
import demjson

import pandas as pd
import pandas_gbq as gbq
import numpy as np
import csv
import re
import requests
import math

import google.cloud
from google.colab import drive
from google.oauth2 import service_account
from google.colab import auth
from google.cloud import bigquery

from web3 import Web3

In [2]:
# authenticate in order to use BigQuery and Cloud Storage in a jupyter notebook running on a local machine
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
# check the table id under 'crypto-project-30940/amm'
project_id = 'crypto-project-30940'
client = bigquery.Client(project="crypto-project-30940")
dataset_ref = client.dataset("result", project="crypto-project-30940")
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

final
results_1_100000
sushi_V2Router02
sushi_decode_error
sushi_decode_result
sushi_liquidity_result
sushi_swap_result


#  Decode V2Router02 contract

In [5]:
# find smart contract abi by using etherscan 
infura_url = "https://mainnet.infura.io/v3/2fa59413591b4e39af06a81fd925c5b6"
web3 = Web3(Web3.HTTPProvider(infura_url))

# uniswap
V2Router02_contract = '0x7a250d5630b4cf539739df2c5dacb4c659f2488d'.lower()
# sushiswap
# V2Router02_contract = '0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f'.lower()
api_key = 'TRDHFVRNF798GQ9S8KJ1JCBT4S8E5SK23Y' # Xiaofeng's private API key of etherscan

V2Router02_address = Web3.toChecksumAddress(V2Router02_contract)

abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address="+ V2Router02_contract + "&apikey=" + api_key
abi = json.loads(requests.get(abi_endpoint).text)
# uniswap_abi or sushiswap_abi
contract = web3.eth.contract(address = V2Router02_address, abi = abi["result"])



In [6]:
# display dataframes as interactive tables
%load_ext google.colab.data_table

In [7]:
# since the data is very large, let us just select 'hash' and 'input' from big query
%%bigquery --project crypto-project-30940 V2Router02_input
  SELECT `hash`, from_address, block_timestamp, receipt_status, input,
   FROM `crypto-project-30940.amm.uniswap_V2Router02_transactions_by20201130`
   limit 1000000 offset 0

In [9]:
def find_input(input_value):
  try: 
    output_value = contract.decode_function_input(input_value)
  except:
    output_value = np.nan

  return output_value

def find_function(decode_str):
  if decode_str is np.nan:
    output_func = np.nan
  else:
    output_func = re.findall('\ (.*)\(', str(decode_str[0]))[0]
  
  return output_func

def find_params(decode_str):
  if decode_str is np.nan:
    output_params = np.nan
  else:
    output_params = dict(decode_str[1])
  
  return output_params

In [10]:
# decode the 'input'
V2Router02_input['decode'] = V2Router02_input['input'].apply(lambda x: find_input(x))
V2Router02_input['function'] = V2Router02_input['decode'].apply(lambda x: find_function(x))
V2Router02_input['params'] = V2Router02_input['decode'].apply(lambda x: find_params(x))

### Decode swap transactions

In [ ]:
## Maybe we need to double check the multiplecalss?? or it is only on V3

In [11]:
# function to get the value of the dictionary based on index, and return np.nan if not exist.
def decode_input(input_params, check_index):
  try:
    output_value = input_params[check_index]
    if check_index in ['amountInMax', 'amountIn', 'amountOutMin', 'amountOut', 'deadline',
                       'amountADesired', 'amountBDesired', 'amountAMin', 'amountBMin', 'liquidity']:
      output_value = str(output_value)
    if check_index == 'path':
      output_value = list(map(str.lower, output_value))
    if check_index in ['to', 'tokenA', 'tokenB', 'token']:
      output_value = output_value.lower()
  except: 
    output_value = np.nan
    
  return output_value


In [12]:
# select swap data
V2Router02_swap = V2Router02_input[V2Router02_input['function'].str.find('swap') != -1]

In [13]:
len(V2Router02_swap)

927038

In [14]:
# decode input for swap file
V2Router02_swap['amountInMax'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'amountInMax'))
V2Router02_swap['amountIn'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'amountIn'))
V2Router02_swap['amountOutMin'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'amountOutMin'))
V2Router02_swap['amountOut'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'amountOut'))
V2Router02_swap['path'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'path'))
V2Router02_swap['to'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'to'))
V2Router02_swap['deadline'] = V2Router02_swap['params'].apply(lambda x: decode_input(x, 'deadline'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [15]:
# function to decode path
def find_num_tokens(input_path):
  if input_path is np.nan:
    output_num_tokens = np.nan
  else:
    output_num_tokens = len(input_path)

  return output_num_tokens


def decode_path(input_path, path_num):
  if input_path is np.nan:
    output_path = np.nan
  elif path_num > len(input_path):
    output_path = np.nan
  else:
    output_path = input_path[path_num-1]

  return output_path

In [16]:
# decode path for swap file
V2Router02_swap['num_tokens'] = V2Router02_swap['path'].apply(lambda x: find_num_tokens(x))
V2Router02_swap['path1'] = V2Router02_swap['path'].apply(lambda x: decode_path(x, 1))
V2Router02_swap['path2'] = V2Router02_swap['path'].apply(lambda x: decode_path(x, 2))
V2Router02_swap['path3'] = V2Router02_swap['path'].apply(lambda x: decode_path(x, 3))
V2Router02_swap['path4'] = V2Router02_swap['path'].apply(lambda x: decode_path(x, 4))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [17]:
%%bigquery --project crypto-project-30940 tokens_info
  SELECT address, symbol, 
  FROM `crypto-project-30940.amm.tokens`

In [18]:
# making address and token into a dictionary. I believe it would be easier to check
tokens_dict = dict(zip(tokens_info.address, tokens_info.symbol))


In [19]:
# function to check the symbol
def check_token_symbol(token_address, token_dict):
  if token_address is np.nan:
    output_token_symbol = np.nan
  else:
    try:
      output_token_symbol = token_dict[token_address]
    except:
      output_token_symbol = np.nan

  return output_token_symbol


In [20]:
# find the token symbol for the corresponding token address
V2Router02_swap['token1'] = V2Router02_swap['path1'].apply(lambda x: check_token_symbol(x, tokens_dict))
V2Router02_swap['token2'] = V2Router02_swap['path2'].apply(lambda x: check_token_symbol(x, tokens_dict))
V2Router02_swap['token3'] = V2Router02_swap['path3'].apply(lambda x: check_token_symbol(x, tokens_dict))
V2Router02_swap['token4'] = V2Router02_swap['path4'].apply(lambda x: check_token_symbol(x, tokens_dict))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [21]:
# remove some intermediate columns
V2Router02_swap = V2Router02_swap.drop(columns=['decode', 'input', 'params'])
V2Router02_swap = V2Router02_swap[V2Router02_swap['function'].notnull()]

In [22]:
len(V2Router02_swap)

926751

In [23]:
V2Router02_swap.to_gbq('result.uniswap_swap_result_YX', project_id='crypto-project-30940',if_exists='append')


1it [02:49, 169.45s/it]


### Decode liquidity transactions

In [24]:
# select liquidity data
V2Router02_liquidity = V2Router02_input[V2Router02_input['function'].str.find('Liquidity') != -1]
V2Router02_liquidity = V2Router02_liquidity[V2Router02_liquidity['function'].notnull()]

In [25]:
def decode_input_liquidity(input_function, input_params, names):
  if str(input_function).find('ETH') == -1:
    try:
      output_value = decode_input(input_params, names)

    except: 
      output_value = np.nan
  else:
    try: 
      if names == 'tokenB':
        names = 'token'
      elif names == 'amountBDesired':
        names = 'amountTokenDesired'
      elif names == 'amountBMin':
        names = 'amountTokenMin'
      elif names == 'amountADesired':
        names = 'amountETHDesired'
      elif names == 'amountAMin':
        names = 'amountETHMin'
      output_value = decode_input(input_params, names)

      if names == 'tokenA':
        output_value = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
    
    except:
      output_value = np.nan

  return output_value
      

In [26]:
V2Router02_liquidity['tokenA_address'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'tokenA'), axis=1)
V2Router02_liquidity['tokenB_address'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'tokenB'), axis=1)
V2Router02_liquidity['amountADesired'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'amountADesired'), axis=1)
V2Router02_liquidity['amountBDesired'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'amountBDesired'), axis=1)
V2Router02_liquidity['amountAMin'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'amountAMin'), axis=1)
V2Router02_liquidity['amountBMin'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'amountBMin'), axis=1)
V2Router02_liquidity['liquidity'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'liquidity'), axis=1)
V2Router02_liquidity['to'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'to'), axis=1)
V2Router02_liquidity['deadline'] = V2Router02_liquidity.apply(lambda x: decode_input_liquidity(x.function, x.params, 'deadline'), axis=1)

In [27]:
# find the token symbol for the corresponding token address
V2Router02_liquidity['tokenA_sysmbol'] = V2Router02_liquidity['tokenA_address'].apply(lambda x: check_token_symbol(x, tokens_dict))
V2Router02_liquidity['tokenB_sysmbol'] = V2Router02_liquidity['tokenB_address'].apply(lambda x: check_token_symbol(x, tokens_dict))

In [ ]:
#V2Router02_liquidity[V2Router02_liquidity['tokenB_sysmbol'].notnull()]

In [28]:
V2Router02_liquidity = V2Router02_liquidity.drop(columns=['decode', 'input', 'params'])

In [29]:
len(V2Router02_liquidity)

72961

In [30]:
V2Router02_liquidity.to_gbq('result.uniswap_liquidity_result_YX', project_id='crypto-project-30940',if_exists='append')


1it [00:13, 13.78s/it]


In [ ]:
# check
# V2Router02_liquidity[V2Router02_liquidity['from_address'] == '0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780']
# V2Router02_liquidity.iloc[1940]
# V2Router02_swap.iloc[1940][0]

,hash,from_address,block_timestamp,receipt_status,function,tokenA_address,tokenB_address,amountADesired,amountBDesired,amountAMin,amountBMin,liquidity,to,deadline,tokenA_sysmbol,tokenB_sysmbol
76836,0x134bf9db48b1aafe93d409e3ba1489e47822767b6ca4...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-19 19:41:44+00:00,1,removeLiquidityETHWithPermit,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,NaN,NaN,264986657521972630,4411467706445154,21375930030959789,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605816054,WETH,NaN
76960,0x7271bf135c19bff261daab19e02b4f6d486676d75a13...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-19 19:09:32+00:00,1,addLiquidityETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,NaN,647901895077386018109,2075038752381912207,615506800323516717203,NaN,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605814166,WETH,NaN
89495,0x8fee4460ed3c2d4d5513421c7752b97228d0ed5fe062...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-17 15:11:28+00:00,1,addLiquidity,0x04fa0d235c4abf4bcf4787af4cf447de572ef828,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,61473368893425958408,1076317062736528769,61166002048958828615,1070935477422846125,NaN,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605627039,NaN,WETH
90406,0x547a5651369eb23981f7cc74b7d2c76d22f1385e9a25...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-17 15:51:33+00:00,1,addLiquidity,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5253373324672723,223682937263471231,5227106458049359,222564522577153874,NaN,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605629474,NaN,WETH
93795,0x44f646c1153c109c47cd223f121e870684d6822ab593...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-14 12:20:27+00:00,1,addLiquidityETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xcaaa93712bdac37f736c323c93d4d5fdefcc31cc,NaN,5120032102456436732925,950000000000000000,4864030497333614896278,NaN,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605357604,WETH,CRD
157577,0xb20e0e7a65dc19abc451c85197e21d5d01cee45861bc...,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,2020-11-15 18:14:39+00:00,1,addLiquidityETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,NaN,154976377977677540476,415697497798979428,147227559078793663452,NaN,0x01c51aa0ee555b8f6bb6358f9042ac6fb55c9780,1605465264,WETH,NaN
